# Notas

Este notebook é umas versão traduzida e adaptada do trabalho disponibilizado no GitHub oficial [NVidia/NeMo](https://colab.research.google.com/github/NVIDIA/NeMo/blob/stable/tutorials/asr/ASR_CTC_Language_Finetuning.ipynb#scrollTo=FnmVqx8aegwR)

# Setup

In [ ]:
# Instala as dependências
!pip install wget
!apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
!pip install unidecode
!pip install matplotlib>=3.3.2

## Instala o pacote NeMo
BRANCH = 'r1.9.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# Ajustando modelos CTC em portugues-br

Nos repositório NeMo é possível encontrar as partes anteriores deste tutorial. Este tutorial aprofundara somente as etapas essenciais, sendo elas:

* Pré-processamento de dados
* Preparação dos tokens
* Treinamento de um modelo CTC de codificação de caracteres

Neste tutorial (limitado pela computação e armazenamento disponíveis nos ambientes Colab), treinaremos um modelo ASR em português no conjunto de dados [Mozzila Commom Voces 9.0](https://commonvoice.mozilla.org/en). 

**Observação**: é aconselhável revisar o diagrama de fluxo de execução para modelos ASR para configurar corretamente o modelo antes do ajuste fino - [Exemplos ASR CTC](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/asr_ctc/README.md)


In [ ]:
import os
import glob
import subprocess
import tarfile
import wget
import copy
from omegaconf import OmegaConf, open_dict

In [ ]:
data_dir = 'datasets/'

if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)

if not os.path.exists("scripts"):
    os.makedirs("scripts")

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

# Download do conjunto de dados

Usaremos uma versão modificada do script disponibilizado no diretório `scripts` do projeto NeMo para baixar e preparar o conjunto de dados [Mozilla Common Voice (MCV)](https://commonvoice.mozilla.org/pt) para português.

O script de preparação de dados fará o download dos arquivos de áudio e suas respectivas transcrições e, em seguida, processará o áudio no formato wave mono-channel de 16 kHz para que possam ser facilmente utilizados para treinar modelos ASR.



In [ ]:
if not os.path.exists("scripts/get_commonvoice_data.py"):
    !wget -P scripts/ https://gist.githubusercontent.com/DominguesM/dcb5e0752abb59f2aa12183729815fcc/raw/9a48e5d64f3a1a848f873100e433c7edd54a5b30/get_commonvoice_data.py

In [ ]:
VERSION = "cv-corpus-9.0-2022-04-27"
LANGUAGE = "pt"

In [ ]:
tokenizer_dir = os.path.join('tokenizers', LANGUAGE)
manifest_dir = os.path.join('manifests', LANGUAGE)

In [ ]:
# Se algo der errado durante o processamento de dados, descomente a linha a seguir para excluir o conjunto de dados em cache
# !rm -rf datasets/$LANGUAGE

In [ ]:
!python scripts/get_commonvoice_data.py \
    --data_root "datasets/$LANGUAGE/" \
    --manifest_dir=$manifest_dir \
    --sample_rate=16000 \
    --n_channels=1 \
    --version=$VERSION \
    --language=$LANGUAGE \
    --log \
    --files_to_process 'train.tsv' 'dev.tsv' 'test.tsv'

Agora que o conjunto de dados foi baixado, vamos preparar alguns caminhos para acessar facilmente os arquivos de manifesto das partições train, dev e test.

In [ ]:
train_manifest = f"{manifest_dir}/commonvoice_train_manifest.json"
dev_manifest = f"{manifest_dir}/commonvoice_dev_manifest.json"
test_manifest = f"{manifest_dir}/commonvoice_test_manifest.json"

# Preparando o conjunto de dados para treinamento

Antes de começarmos a treinar o modelo nos arquivos de manifesto não processados acima, precisamos analisar os dados. O pré-processamento de dados talvez seja a tarefa mais essencial e geralmente requer conhecimento moderado no idioma.

Embora possamos tecnicamente usar os manifestos acima para treinar um modelo, os resultados seriam potencialmente péssimos. Vamos nos aprofundar um pouco mais nos desafios que esse conjunto de dados representa para nossos modelos.

**Nota**: O pré-processamento feito neste corpus é feito especificamente para reduzir a ambiguidade nas transcrições, devido à quantidade minúscula de dados que possuímos. Com dados suficientes, os modelos discutidos aqui poderiam aprender bem, mesmo sem esse pré-processamento pesado.

## Utilitários de manifesto

Primeiro, construímos alguns utilitários para ler e gravar arquivos de manifesto

In [ ]:
# Manifest Utils
from tqdm.auto import tqdm
import json

def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest


def write_processed_manifest(data, original_path):
    original_manifest_name = os.path.basename(original_path)
    new_manifest_name = original_manifest_name.replace(".json", "_processed.json")

    manifest_dir = os.path.split(original_path)[0]
    filepath = os.path.join(manifest_dir, new_manifest_name)
    with open(filepath, 'w') as f:
        for datum in tqdm(data, desc="Writing manifest data"):
            datum = json.dumps(datum)
            f.write(f"{datum}\n")
    print(f"Finished writing manifest: {filepath}")
    return filepath

In [ ]:
train_manifest_data = read_manifest(train_manifest)
dev_manifest_data = read_manifest(dev_manifest)
test_manifest_data = read_manifest(test_manifest)

Em seguida, extraímos apenas o corpus de texto do manifesto.

In [ ]:
train_text = [data['text'] for data in train_manifest_data]
dev_text = [data['text'] for data in dev_manifest_data]
test_text = [data['text'] for data in test_manifest_data]

## Conjunto de caracteres

Vamos calcular o conjunto de caracteres - que é o conjunto de tokens exclusivos que existem nos manifestos de texto.

In [ ]:
from collections import defaultdict

def get_charset(manifest_data):
    charset = defaultdict(int)
    for row in tqdm(manifest_data, desc="Computing character set"):
        text = row['text']
        for character in text:
            charset[character] += 1
    return charset

In [ ]:
train_charset = get_charset(train_manifest_data)
dev_charset = get_charset(dev_manifest_data)
test_charset = get_charset(test_manifest_data)

Conta o número de tokens exclusivos que existem nos conjuntos de dados

In [ ]:
train_dev_set = set.union(set(train_charset.keys()), set(dev_charset.keys()))
test_set = set(test_charset.keys())

In [ ]:
print(f"Number of tokens in train+dev set : {len(train_dev_set)}")
print(f"Number of tokens in test set : {len(test_set)}")

## Contar o número de tokens [OOV](https://medium.com/analytics-vidhya/handling-out-of-vocabulary-words-in-natural-language-processing-based-on-context-4bbba16214d5) no conjunto de teste

Dado que existe um número tão grande de tokens no conjunto train e dev, vamos garantir que não haja tokens discrepantes no conjunto de teste.

In [ ]:
# OOV tokens in test set
train_test_common = set.intersection(train_dev_set, test_set)
test_oov = test_set - train_test_common
print(f"Number of OOV tokens in test set : {len(test_oov)}")
print()
print(test_oov)

## Remova os tokens OOV do conjunto de teste

Anteriormente, contamos o conjunto de tokens OOV que existem no conjunto de teste, mas não no conjunto de treinamento ou desenvolvimento. Agora, vamos removê-los.

In [ ]:
all_tokens = set.union(train_dev_set, test_set)
print(f"Original train+dev+test vocab size : {len(all_tokens)}")

extra_words = set(test_oov)
train_token_set = all_tokens - extra_words
print(f"New train vocab size : {len(train_token_set)}")

## Normalização de acentos

A lingua portuguesa é composta por palavras com e sem acentos. Normalmente, é essencial capturar as diferenças acústicas entre os tokens com e sem acentuação. No entando, em conjunto de dados pequenos pode reduzir a capacidade do modelo de aprender e desambiguar entre os dois tipos de tokens.

Abaixo, oferecemos um sinalizados para substituir uma letra com acentuação para uma sem acentuação (normalizada).

In [ ]:
#@title Unicode normalization
perform_unicode_normalization = False #@param ["True", "False"] {type:"raw"}
PERFORM_UNICODE_NORMALIZATION = bool(perform_unicode_normalization)

In [ ]:
import unicodedata
def process_text(text):
    normalized_text = unicodedata.normalize(u'NFKD', text).encode('ascii', 'ignore').decode('utf8')
    return normalized_text

In [ ]:
if PERFORM_UNICODE_NORMALIZATION:
    normalized_train_token_set = set()
    for token in train_token_set:
        normalized_token = process_text(str(token))
        normalized_train_token_set.update(normalized_token)
        
    print(f"After unicode normalization, number of train tokens : {len(normalized_train_token_set)}")
else:
    normalized_train_token_set = train_token_set
    

## Processar tokens de caracteres especiais

Existem vários tokens que não correspondem com precisão a uma característica acústica. Alguns exemplos são várias vírgulas e o ponto final. Pense dessa maneira, a menos que cada frase termine com um ponto (e isso é incomum - já que os conjuntos de dados de treinamento geralmente são compostos de pequenos trechos de áudio de conversas mais longas), então um modelo não tem contexto suficiente para determinar quando terminar uma frase apenas do snippet que foi fornecido.

Como tal, removemos vários tokens especiais, como vírgulas, pontos de interrogação, pontos, aspas e alguns tokens especiais usados às vezes em texto em português.

In [ ]:
# Preprocessing steps
import re
import unicodedata

chars_to_ignore_regex = '[\!\"\&\'\,\-\.\:\;\?\«\´\»\“\”]'  # remove special character tokens

def remove_special_characters(data):
    replace_list = [
      ("ü","u"),
      ("š","s"),
      ("è","e"),
      ("ž","z"),
      ("ñ","n"),
      ("â","a"),
      ("à","á"),
      ("â","a"),
      ("ô","o"),
      ("ú","u"),

    ]
    for special_char in replace_list:
        data["text"] = data["text"].replace(*special_char)

    data["text"] = re.sub(chars_to_ignore_regex, '', data["text"]).lower().strip()
    return data

def remove_extra_word(data):
    data["text"] = data["text"].replace("è","e")
    return data

def normalize_unicode(data):
    # perform unicode normalization (if it was requested)
    if PERFORM_UNICODE_NORMALIZATION:
        text = data['text']
        data['text'] = process_text(text)
    return data

## Normalizações no conjunto de treino

Agora que temos as funções necessárias para limpar as transcrições, vamos criar um pequeno pipeline para limpar o manifesto e escrever novos manifestos para nós. Para simplificar, um pipeline sequencial simples será suficiente para nosso caso de uso.

In [ ]:
# Processing pipeline
def apply_preprocessors(manifest, preprocessors):
    for processor in preprocessors:
        for idx in tqdm(range(len(manifest)), desc=f"Applying {processor.__name__}"):
            manifest[idx] = processor(manifest[idx])

    print("Finished processing manifest !")
    return manifest

In [ ]:
# List of pre-processing functions
PREPROCESSORS = [
    remove_special_characters,
    remove_extra_word,
    normalize_unicode,
]

In [ ]:
# Load manifests
train_data = read_manifest(train_manifest)
dev_data = read_manifest(dev_manifest)
test_data = read_manifest(test_manifest)

# Apply preprocessing
train_data_processed = apply_preprocessors(train_data, PREPROCESSORS)
dev_data_processed = apply_preprocessors(dev_data, PREPROCESSORS)
test_data_processed = apply_preprocessors(test_data, PREPROCESSORS)

# Write new manifests
train_manifest_cleaned = write_processed_manifest(train_data_processed, train_manifest)
dev_manifest_cleaned = write_processed_manifest(dev_data_processed, dev_manifest)
test_manifest_cleaned = write_processed_manifest(test_data_processed, test_manifest)


## Conjunto de caracteres final

Após pré-processar o conjunto de dados, vamos recuperar o conjunto de caracteres final usado para treinar os modelos.

In [ ]:
train_manifest_data = read_manifest(train_manifest_cleaned)
train_charset = get_charset(train_manifest_data)

dev_manifest_data = read_manifest(dev_manifest_cleaned)
dev_charset = get_charset(dev_manifest_data)

train_dev_set = set.union(set(train_charset.keys()), set(dev_charset.keys()))

In [ ]:
# exemplo de dados
train_manifest_data[0]

In [ ]:
print(f"Number of tokens in preprocessed train+dev set : {len(train_dev_set)}")

# Modelo CTC de Codificação de Caracteres

Agora que temos um conjunto de dados processado, podemos começar a treinar um modelo ASR nesse conjunto de dados. A seção a seguir detalha como preparamos um modelo CTC que utiliza um esquema de codificação de caracteres.

Esta seção utilizará um [QuartzNet 15x5](https://arxiv.org/abs/1910.10261) pré-treinado, que foi treinado em aproximadamente 7.000 horas de modelo básico de fala em inglês. Vamos modificar a camada do decodificador (alterando assim o vocabulário do modelo) e depois treinar por um pequeno número de `epochs`.

In [ ]:
char_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_quartznet15x5", map_location='cpu')

## Atualizar o vocabulário

Alterar o vocabulário de um modelo ASR de codificação de caracteres é tão simples quanto passar a lista de novos tokens que compõem o vocabulário como entrada para `change_vocabulary()`.

In [ ]:
char_model.change_vocabulary(new_vocabulary=list(train_dev_set))

## Treinamento em idiomas com poucos recursos

Se a quantidade de dados de treinamento ou recursos computacionais disponíveis forem limitados, pode ser útil congelar o módulo codificador da rede e treinar apenas a camada final do decodificador. Isso também é útil nos casos em que a memória da GPU é insuficiente para treinar uma rede grande ou nos casos em que o modelo pode se ajustar demais devido ao seu tamanho.

-------

Nos casos em que há dados suficientes disponíveis - e "suficiente" depende da complexidade do idioma - é aconselhável treinar o codificador também para obter a melhor transcrição possível. Quando dizemos que o suficiente é relativo ao idioma, notamos que alguns idiomas podem obter pontuações razoáveis ​​com algumas centenas de horas de fala transcrita, enquanto alguns idiomas exigem vários milhares de horas.

------

Também é importante notar que, se o idioma permanecer o mesmo e algum domínio específico do texto precisar ser adaptado para ASR, geralmente é mais fácil adicionar um modelo de idioma específico de domínio para orientar o modelo genérico de ASR do que tentar ajustar um modelo ASR completo em dados limitados desse domínio específico.


In [ ]:
#@title Freeze Encoder { display-mode: "form" }
freeze_encoder = False #@param ["False", "True"] {type:"raw"}
freeze_encoder = bool(freeze_encoder)

### Frozen Encoder - Unfrozen Batch Normalization

Congelar o codificador geralmente é útil para limitar a computação e permitir um treinamento mais rápido; no entanto, em muitos experimentos, congelar o codificador em sua totalidade geralmente impedirá que um modelo aprenda em linguagens de poucos recursos.

Para permitir que um modelo de codificador congelado aprenda um novo idioma de forma estável, descongelamos as camadas de normalização de lote no codificador. Além disso, se o modelo contiver submódulos "SqueezeExcite", também os descongelamos.

Ao fazer isso, notamos que tais modelos são treinados adequadamente e obtêm pontuações respeitáveis mesmo em linguagens com recursos severamente limitados.

------

**Observação**: Esse fenômeno desaparece quando há dados suficientes disponíveis (nesse caso, todo o codificador também pode ser treinado). Portanto, é aconselhável descongelar o codificador quando houver dados suficientes disponíveis.


In [ ]:
import torch
import torch.nn as nn

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

In [ ]:
if freeze_encoder:
    char_model.encoder.freeze()
    char_model.encoder.apply(enable_bn_se)
    logging.info("Model encoder has been frozen, and batch normalization has been unfrozen")
else:
    char_model.encoder.unfreeze()
    logging.info("Model encoder has been un-frozen")

## Atualizar configuração

Cada modelo NeMo tem uma configuração embutida nele, que pode ser acessada via `model.cfg`. Em geral, esta é a configuração que foi usada para construir o modelo.

Para modelos pré-treinados, essa configuração geralmente representa a configuração usada para construir o modelo quando ele foi treinado. Um bom benefício para essa configuração incorporada é que podemos redefini-la para configurar novos `data loaders`, `optimizers`, `schedulers` e até mesmo `data augmentation`!

### Atualizando o conjunto de caracteres do modelo

A etapa mais importante para preparar modelos de codificação de caracteres para ajuste fino é atualizar o conjunto de caracteres do modelo. Lembre-se - o modelo foi treinado em alguma linguagem com algum conjunto de dados específico que tinha um determinado conjunto de caracteres. Os conjuntos de caracteres raramente permaneceriam os mesmos entre o treinamento e o ajuste fino (embora ainda seja possível).

Cada modelo de codificação de caracteres tem um atributo `model.cfg.labels`, que pode ser substituído via OmegaConf.

In [ ]:
char_model.cfg.labels = list(train_dev_set)

Agora, criamos uma cópia de trabalho da configuração do modelo e a atualizamos conforme necessário.

In [ ]:
cfg = copy.deepcopy(char_model.cfg)

### Configurando o data loader

Agora que o conjunto de caracteres do modelo foi atualizado, vamos preparar o modelo para utilizar o novo conjunto de caracteres nos `data loaders`. Observe que isso é crucial para que os dados produzidos durante o treinamento/validação correspondam ao novo conjunto de caracteres e os tokens sejam codificados/decodificados corretamente.

**Nota**: Um parâmetro de configuração importante é `normalize_transcripts` e `parser`. Existem alguns analisadores que são usados para linguagens específicas para modelos baseados em caracteres - atualmente apenas `en` é suportado. Esses analisadores irão pré-processar o texto com o analisador de idiomas fornecido. No entanto, para outras linguagens, é aconselhável definir explicitamente `normalize_transcripts = False` - o que impedirá o analisador de processar texto.

In [ ]:
# Setup train, validation, test configs
with open_dict(cfg):    
    # Train dataset  (Concatenate train manifest cleaned and dev manifest cleaned)
    cfg.train_ds.manifest_filepath = f"{train_manifest_cleaned},{dev_manifest_cleaned}"
    cfg.train_ds.labels = list(train_dev_set)
    cfg.train_ds.normalize_transcripts = False
    cfg.train_ds.batch_size = 32
    cfg.train_ds.num_workers = 8
    cfg.train_ds.pin_memory = True
    cfg.train_ds.trim_silence = True

    # Validation dataset  (Use test dataset as validation, since we train using train + dev)
    cfg.validation_ds.manifest_filepath = test_manifest_cleaned
    cfg.validation_ds.labels = list(train_dev_set)
    cfg.validation_ds.normalize_transcripts = False
    cfg.validation_ds.batch_size = 8
    cfg.validation_ds.num_workers = 8
    cfg.validation_ds.pin_memory = True
    cfg.validation_ds.trim_silence = True

In [ ]:
# setup data loaders with new configs
char_model.setup_training_data(cfg.train_ds)

char_model.setup_multiple_validation_data(cfg.validation_ds)

### Configurando o `optimizer` e o `scheduler`

Ao ajustar os modelos de caracteres, geralmente é aconselhável usar uma taxa de aprendizado mais baixa e `warmup` reduzido. Uma taxa de aprendizado reduzida ajuda a preservar os pesos pré-treinados do codificador. Como o conjunto de dados de ajuste fino geralmente é menor que o conjunto de dados de treinamento original, as etapas de `warmup` seriam demais para o conjunto de dados de ajuste fino menor.

-----
**Observação**: Ao congelar o codificador, é possível usar a taxa de aprendizado original conforme o modelo foi treinado. A taxa de aprendizado original pode ser usada porque o codificador está congelado, portanto, a taxa de aprendizado é usada apenas para otimizar o decodificador. No entanto, uma taxa de aprendizado muito alta ainda desestabilizaria o treinamento, mesmo com um codificador congelado.

**Warmup**: Geralmente significa que você usa uma taxa de aprendizado muito baixa para um número definido de etapas de treinamento (warmup steps). Após as etapas de `warmup` você usa sua taxa de aprendizado "regular" ou `learning rate scheduler`. Você também pode aumentar gradualmente sua taxa de aprendizado ao longo do número de etapas de aquecimento.

In [ ]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(char_model.cfg.optim))

In [ ]:
with open_dict(char_model.cfg.optim):
    char_model.cfg.optim.lr = 0.01
    char_model.cfg.optim.betas = [0.95, 0.5]  # from paper
    char_model.cfg.optim.weight_decay = 0.001  # Original weight decay
    char_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
    char_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
    char_model.cfg.optim.sched.min_lr = 1e-5

### Configurando augmentation

Lembre-se de que o modelo foi treinado em vários milhares de horas de dados, portanto, a regularização fornecida a ele pode não se adequar ao conjunto de dados atual. Podemos facilmente mudá-lo como acharmos melhor.

-----

Você pode notar que utilizamos `char_model.from_config_dict()` para criar um novo objeto SpectrogramAugmentation e atribuí-lo diretamente no lugar do aumento anterior. Esta é geralmente a sintaxe a ser seguida sempre que você notar uma tag `_target_` na configuração da configuração interna de um modelo.

-----
**Observação**: para linguagens com poucos recursos, pode ser melhor aumentar o aumento via SpecAugment para reduzir o overfitting. No entanto, isso pode, por sua vez, tornar muito difícil para o modelo treinar em um pequeno número de `epochs`.

**Data augmentation**: O aumento de dados na análise de dados são técnicas usadas para aumentar a quantidade de dados adicionando cópias ligeiramente modificadas de dados já existentes ou dados sintéticos recém-criados a partir de dados existentes.

In [ ]:
print(OmegaConf.to_yaml(char_model.cfg.spec_augment))

In [ ]:
# with open_dict(char_model.cfg.spec_augment):
#   char_model.cfg.spec_augment.freq_masks = 2
#   char_model.cfg.spec_augment.freq_width = 25
#   char_model.cfg.spec_augment.time_masks = 2
#   char_model.cfg.spec_augment.time_width = 0.05

char_model.spec_augmentation = char_model.from_config_dict(char_model.cfg.spec_augment)

## Configurar Métricas

Originalmente, o modelo foi treinado em um corpus de conjunto de dados em inglês. Ao calcular a taxa de erro de palavras (`WER - Word Error Rate`), podemos facilmente usar o token "espaço" como separador para limites de palavras. Por outro lado, alguns idiomas, como japonês e mandarim, não usam tokens de "espaço", optando por diferentes maneiras de anotar o final da palavra.

Nos casos em que o token "espaço" não é usado para denotar um limite de palavra, podemos usar a métrica Taxa de erro de caractere, que calcula a distância de edição em um nível de token em vez de em um nível de palavra.

No caso do portugues podemos utilizar `WER`.

In [ ]:
#@title Metric
use_cer = False #@param ["False", "True"] {type:"raw"}
log_prediction = True #@param ["False", "True"] {type:"raw"}

In [ ]:
char_model._wer.use_cer = use_cer
char_model._wer.log_prediction = log_prediction

## Setup Trainer and Experiment Manager

E é isso! Agora podemos treinar o modelo simplesmente usando o Pytorch Lightning Trainer e o NeMo Experiment Manager.

Para fins de demonstração, o número de épocas é mantido intencionalmente baixo. Resultados razoáveis podem ser obtidos em cerca de 100 épocas (aproximadamente 21 horas utilizando as GPUs do Google Colab).

**Nota**: O treino com 26 horas de áudio em português demorou 11 horas com 50 epochs.


In [ ]:
import torch
import pytorch_lightning as ptl

if torch.cuda.is_available():
    accelerator = 'gpu'
else:
    accelerator = 'cpu'

EPOCHS = 50  # 100 epochs would provide better results, but would take an hour to train

trainer = ptl.Trainer(devices=1, 
                      accelerator=accelerator, 
                      max_epochs=EPOCHS, 
                      accumulate_grad_batches=1,
                      enable_checkpointing=False,
                      logger=False,
                      log_every_n_steps=5,
                      check_val_every_n_epoch=10)

# Setup model with the trainer
char_model.set_trainer(trainer)

# Finally, update the model's internal config
char_model.cfg = char_model._cfg

In [ ]:
# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/lang-{LANGUAGE}/',
    name=f"ASR-Char-Model-Language-{LANGUAGE}",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

config = OmegaConf.structured(config)

logdir = exp_manager.exp_manager(trainer, config)

In [ ]:
try:
    from google import colab
    COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
    COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
    %load_ext tensorboard
    %tensorboard --logdir /content/experiments/lang-$LANGUAGE/ASR-Char-Model-Language-$LANGUAGE/
else:
    print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
%%time
trainer.fit(char_model)

## Resultados

**Epochs**: 50

**Horas de treino**: 11 horas

* Test WER on Common Voice Portuguese: 49%
* Test CER on Common Voice Portuguese: 18%


# Salve o modelo final

Por fim, podemos salvar um ponto de verificação (`checkpoint`) (que pode ser baixado na guia do navegador de arquivos em um ambiente colab).

In [ ]:
save_path = f"./Model-{LANGUAGE}.nemo"
char_model.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")